# Dataset Tranformation

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

Pada bagian ini kita melakukan sedikit modifikasi:

- Pada bagian def `__init__`, kita menambahkan self.transform = transform, juga tidak lupa pada bagian argumen transform
- Pada bagian def `__getitem__`, kita menambahkan self.transform

In [2]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        xy = np.loadtxt('./11_wine_dataset/wine.csv', delimiter=',', dtype=np.float32,
                        skiprows=1)  # skip row 1 karena row 1 adalah header
        
        self.n_samples = len(xy)
        
        self.x = xy[:, 1:]  # mengabaikan kolom indeks-0 karena kolom tersebut adalah kelasnya
        self.y = xy[:, [0]]
        
        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):
        return self.n_samples

# Membuat class ToTensor()
- Kelas ini berfungsi untuk mengkonversikan nilai masukan dan target ke dalam bentuk tensor.
- `__call__` adalah method bawaan dari python yang memungkinkan pengguna membuat kelas yang instancenya dapat dipanggil sebagai fungsi.

In [3]:
class ToTensor():
    def __call__(self, sample):
        x, y = sample
        return torch.from_numpy(x), torch.from_numpy(y)

dataset = WineDataset(transform=None)

Mencoba mengambil sampel data awal. Dan pada bagian ini, tidak ada transformasi yang dilakukan

In [4]:
first_data = dataset[0]
contoh_x, contoh_y = first_data
print(contoh_x, contoh_y)
print(contoh_x.shape, contoh_y.shape)
print(type(contoh_x), type(contoh_y))

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]
(13,) (1,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


# Membuat class MulTransform()
- Kelas ini berfungsi untuk mengalikan input dengan faktor pengali
- Nilai kembalian dari fungsi ini adalah input (x) yang telah dikali dan target (y) yang tetap.

In [5]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        x, y = sample
        x = x * self.factor
        return x, y

# Membuat Beberapa Transformasi Menjadi Satu
- Dapat diimplementasikan dengan torchvision.transforms.Compose
- Tujuannya adalah, kita menggabungkan transformasi yang kita buat pada kelas ToTensor dan MulTransform menjadi sebuah fungsi transformasi

In [6]:
import torchvision

composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset  = WineDataset(transform=composed)

first_data = dataset[0]
contoh_x, contoh_y = first_data
print(contoh_x, contoh_y)
print(contoh_x.shape, contoh_y.shape)
print(type(contoh_x), type(contoh_y))

tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])
torch.Size([13]) torch.Size([1])
<class 'torch.Tensor'> <class 'torch.Tensor'>


Dapat kita amati beberapa hal:

- Setelah menggunakan fungsi transformasi composed, kita akan mendapatkan input dan target yang telah dikonversikan ke dalam bentuk tensor.
- Nilai dari contoh_x dan contoh_y setelah ditransformasi juga berbeda dengan sebelum di transformasikan.
- torchvision.transforms.Compose memungkinkan kita untuk menggabungkan beberapa fungsi transformasi.